In [6]:
from deepface import DeepFace
from imageio import imread

In [ ]:
role_arn = "your_role_arn"

In [8]:
emotion_model = DeepFace.build_model("Emotion")

In [9]:
model_path = '{}/{}/00000001'.format("model/", "sagemaker_demo_model_tensorflow")

emotion_model.save(filepath=model_path)

In [10]:
import tarfile
zipped_model_path = "sagemaker_demo_model.tar.gz"
with tarfile.open(zipped_model_path, "w:gz") as tar:
    tar.add(model_path)


In [3]:
import boto3
bucket_name = "sagemaker-demo-20231108"
region = "ap-northeast-1"

s3_client = boto3.client('s3', region_name=region)
location = {'LocationConstraint': region}


In [4]:
s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=location)
# s3_client.list_buckets()

{'ResponseMetadata': {'RequestId': 'NAEV1PFG8B9TF2T2',
  'HostId': 'SWgvPcBs/nvnD6U9MqwqJPcTG08uUM8H16kZ+bDfaFBniYMLbM+Z2Cuo0ueJ3I2tkG3YuWqKFJg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'SWgvPcBs/nvnD6U9MqwqJPcTG08uUM8H16kZ+bDfaFBniYMLbM+Z2Cuo0ueJ3I2tkG3YuWqKFJg=',
   'x-amz-request-id': 'NAEV1PFG8B9TF2T2',
   'date': 'Tue, 07 Nov 2023 22:59:09 GMT',
   'location': 'http://sagemaker-demo-20231108.s3.amazonaws.com/',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': 'http://sagemaker-demo-20231108.s3.amazonaws.com/'}

In [13]:
s3_client.upload_file(zipped_model_path, bucket_name, zipped_model_path)

In [16]:
from sagemaker.tensorflow import TensorFlowModel

endpoint_name = "sagemakerDemoEndpoint" 
data_url = "s3://{}/{}".format(bucket_name, zipped_model_path)
tensorFlowModel = TensorFlowModel(
    model_data=data_url,
    role=role_arn,
    image_uri="763104351884.dkr.ecr.ap-northeast-1.amazonaws.com/tensorflow-inference:2.13.0-cpu-py310-ubuntu20.04-sagemaker"
)

predictor = tensorFlowModel.deploy(
    initial_instance_count=1, instance_type="ml.c6i.large", endpoint_name=endpoint_name
)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/itsuki/Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/itsuki/Library/Application Support/sagemaker/config.yaml
---!

## invoke Enpoint

In [17]:
from sagemaker.tensorflow import TensorFlowPredictor

predictor = TensorFlowPredictor(endpoint_name)


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/itsuki/Library/Application Support/sagemaker/config.yaml


In [20]:
import cv2

classifier_path = "haarcascade_frontalface_default.xml"
face_classifier_frontal = cv2.CascadeClassifier(classifier_path)

imagePath = 'test_image.jpg'
image_cv2 = cv2.imread(imagePath)
image_cv2 = cv2.cvtColor(image_cv2, cv2.COLOR_BGR2GRAY)
faces_frontal = face_classifier_frontal.detectMultiScale(image_cv2)
(x, y, w, h) = faces_frontal[0]
face_roi = image_cv2[y:y + h, x:x + w]
resized_face = cv2.resize(face_roi, (48, 48), interpolation=cv2.INTER_AREA)
normalized_face = resized_face / 255.0
reshaped_face = normalized_face.reshape(1, 48, 48, 1)
response = predictor.predict(reshaped_face)
predictions = response['predictions']
print(predictions)

[[0.0183952674, 1.11191099e-07, 0.0041606226, 0.551365137, 0.0831287652, 0.00045347563, 0.342496574]]


In [21]:
import numpy as np

emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
emotion_idx = np.array(predictions[0]).argmax()
emotion = emotion_labels[emotion_idx]
print(emotion)

happy
